This is currently the best code for calibrating the tagger timing and saving the offsets.

# A (Possible) Code for Calibrating the Tagger Timing

First, we import the tools we'll need:

In [1]:
import Acqu as aq
import AcquDetector as aqdet
import ROOT
import numpy as np
import json
from collections import OrderedDict
from rootpy.plotting import histogram, Hist2D, Hist, Canvas

Welcome to JupyROOT 6.16/00


Import the data to be analyzed:

In [2]:
inFile = "/scratch/2019-05_Timepix/Timepix_33.dat"
aq.openFile(inFile)

Mk2 Data


Load in the detector file for the Tagger:

In [3]:
aqdet.LoadDetectors(["taggerNewer.json"])

taggerNewer.json


We set up histograms for each of the channels, and then fill them with tagger times from our data:

In [5]:
taggerChannels = aqdet.Channels["Tagger"]     # number of channels in the tagger (368)

global taggTime
taggTime = [None]*taggerChannels                # set up an array of 368 histograms to fill      

for i in range(taggerChannels):               # for each tagger channel
    taggTime[i] = Hist(1300,-500,800)           # set up a histogram for that tagger channel

def plotCalTagger():
    data = aqdet.Calibrate(aq.adcArray)              # calibrate the data
    taggerTimes = aqdet.Arrays["Tagger"]["Time"]     # get the tagger times
    
    for dat in taggerTimes:                          # for each data point
        for time in dat["Time"]:                     # for each tagger time
            chan = dat["channel"]                 # get the current channel        
            taggTime[chan].Fill(time)             # put that time in the histogram for the current channel
                    
    if(aq.eventNo%5000==0):                          # print a processing statement every 5000 events
        print("Events Processed: "+str(aq.eventNo))
    
aq.runFunction(plotCalTagger,0,500000)               # run this process over 500000 events

Events Processed: 5000
Events Processed: 10000
Events Processed: 15000
Events Processed: 20000
Events Processed: 25000
Events Processed: 30000
Events Processed: 35000


KeyboardInterrupt: 

Now we plot the data for the tagger channels and apply a gaussian fit (with an offset) to them:

In [ ]:
tagger_times = ROOT.TFile("tagger_times.root","RECREATE")     # create a root file to put all the histograms into

myFit = ROOT.TF1("myFit","gaus(0)+pol0(3)")                   # make a gaussian fit with an offset/background
myFit.SetParNames("constant","mean","stdev","background")     # label the parameters

badFitChannels = []      # set up an array to remember the empty or near empty histograms

con = [None]*taggerChannels           # set up an array for the constant value
peak = [None]*taggerChannels          # set up an array for the peak locations
sigma = [None]*taggerChannels         # set up an array for the standard deviations of the peaks
bgLevel = [None]*taggerChannels       # set up an array for the background levels

sig = 1.1    # initialize a standard deviation value to try
bg = 100     # initialize a background value to try

for i in range(taggerChannels):              # for each channel in the tagger    
    taggTime[i].SetTitle("Channel "+str(i))    # set the title for the histogram              
    
    if(taggTime[i].GetEntries()>10):                         # if the histogram has entries
        binmax = taggTime[i].GetMaximumBin()                 # find the maximum number of entries in a bin
        pk = taggTime[i].GetXaxis().GetBinCenter(binmax)     # find the bin corresponding to the max
        
        myFit.SetParameter(1,pk)          # try a peak value near the fullest bin
        myFit.SetParameter(2,sig)         # try this standard deviation value for the fit       
        myFit.SetParameter(3,bg)          # try this background value for the fit
        
        taggTime[i].Fit("myFit","S","",-300,600)    # fit the histogram with a gaussian
        
        con[i] = taggTime[i].GetFunction("myFit").GetParameter("constant")           # save the constant value
        peak[i] = taggTime[i].GetFunction("myFit").GetParameter("mean")              # save the peak position
        sigma[i] = taggTime[i].GetFunction("myFit").GetParameter("stdev")            # save the standard deviation
        bgLevel[i] = taggTime[i].GetFunction("myFit").GetParameter("background")     # save the background level
        
        # check to make sure we're getting the fit we want
        if(ROOT.gMinuit.fCstatu!="CONVERGED " or not(120<peak[i]<160) or not(0<sigma[i]<4) or bgLevel[i]<0 or con[i]<0):  
            print("Channel "+str(i)+" fit has failed.")     # if this is printed, see OldTaggerCalib.ipynb for possible while loop
            badFitChannels.append(i)                        # note that this channel was not fit
        
        sig = sigma[i]       # try using this sigma value for the next channel
        bg = bgLevel[i]      # try using this background value for the next channel
       
    else:       # if the histogram is empty                                     
        print("Histogram "+str(i)+" is empty (has "+str(taggTime[i].GetEntries())+" entries).")    # then say so 
        badFitChannels.append(i)               # note that this channel was not fit
     
    taggTime[i].Write("tagger_times")     # write the histogram to a root file for viewing

Shifting the histograms so that the peak of each is at zero (likely not needed - but a good visual tool):

In [ ]:
tagger_calib = ROOT.TFile("tagger_calib.root","RECREATE")    # create a root file to put all the histograms into

histos_calib = [None]*taggerChannels            # set up an array of 368 histograms to fill

for chan in range(taggerChannels):              # for each channel in the tagger
    histos_calib[chan] = Hist(1300,-500,800)    # create a histogram            
    if (peak[chan]!=None):                      # so long as there is a peak value saved
        histos_calib[chan] = Hist(1300,int(-500-peak[chan]),int(800-peak[chan]))      # create a histogram shifted back by the value of the peak      
        for tbin in range(1300):                                                      # for each bin in that channel's data
            histos_calib[chan].SetBinContent(tbin,histos[chan].GetBinContent(tbin))   # give the bin that channel's data, now shifted back by the peak value
     
    histos_calib[chan].SetTitle("Channel "+str(chan))    # set the title for the histogram
    
    histos_calib[chan].Write("tagger_calib")             # write the histogram to a root file for viewing

Next, we must import the json file, change the channel offset values, and then save the array again as a new json file. The location of the peak must first be divided by the scale parameter, since the offset values in the setup file are unscaled values (not converted into ns). This value is then added to the current offset value, since we're fitting the peak to the calibrated data. Note that the channels with empty histograms will not rewrite their offset values.

In [ ]:
calibData = json.load(open("taggerNewer.json"), object_pairs_hook=OrderedDict)     # load the tagger calibration data and keep its original ordering

chan = 0                                    # starting at channel 0
for chanData in calibData["channels"]:      # for each tagger channel's data
    if(chan not in badFitChannels):         # so long as the histogram has data fitted
        chanData["offset"][0] += peak[chan]/chanData["scale"][0]      # change the offset value to the correct one
    chan+=1                                 # move on to the next channel

json.dump(calibData, open("taggerWithOffsets.json", "w"), indent = 2)              # write the changed data to a new json file